In [1]:
import sys,os
os.chdir('../../')
sys.path.append('./src/data')
sys.path.append('./src/events')
sys.path.append('./src/probability')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from DC.importer import *
from DC.processer import *
from DC.main import get_all_events,get_events
from functions import perform_chisq, dc_params_nsi, dc_params
from DC.event_processing import list_of_params_nsi, return_precomputed_nsi, get_deltachi,normalize_events
from scipy.stats import chi2
import pickle
plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams['figure.dpi'] = 100
matplotlib.rc('text', usetex=True)
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')
np.set_printoptions(linewidth=200)

9.824673788013054e-06


In [2]:
ndim = 3
pid = 1
precomputed_events = True
dc_params_nsi['dm_41'] = 0.93
dc_params['dm_41'] = 0.93
emm_range = np.linspace(0,0,1)
s24_range = np.linspace(0,0,1)
emt_range = np.linspace(-1e-2,1e-2,10)
param_list = list_of_params_nsi(dc_params_nsi,s24_range, emm_range,emt_range)

#for p in param_list: # Assert all dicts returned from param_list have precomputed probs.
#    assert is_precomputed_nsi(N=N,ndim=ndim, dict=p,check=False)



param_list = return_precomputed_nsi(pid,ndim,params=param_list)
emm_range = np.sort(np.unique(np.array([p['e_mm'] for p in param_list])))
s24_range = np.sin(2*np.sort(np.unique(np.array([p['theta_24'] for p in param_list]))))**2
emt_range = np.sort(np.unique(np.array([p['e_mt'] for p in param_list])))

print(emm_range)
print(s24_range)
print(emt_range)

#print(param_list[1])

[0.]
[0.]
[-0.01       -0.00777778 -0.00555556 -0.00333333 -0.00111111  0.00111111  0.00333333  0.00555556  0.00777778  0.01      ]


In [3]:
param_list = param_list[0:2]

In [4]:
if not precomputed_events:
    print('Computing events')
    from multiprocessing import Pool
    p = Pool()
    data = [(p, pid,False) for p in param_list]
    H1_events_list = p.starmap(get_all_events, data)
    p.close()
    pickle.dump(H1_events_list,open(f'./pre_computed/H1_DC_{pid}_{len(emm_range)}x{len(emt_range)}_tau_nsi.p','wb'))

    H0_events = get_all_events(params=param_list[0], pid=pid, nsi=True)
    pickle.dump(H0_events,open(f'./pre_computed/H0_DC_{pid}.p','wb'))

H1_events_list = pickle.load(open(f'./pre_computed/H1_DC_{pid}_{len(emm_range)}x{len(emt_range)}_tau_nsi.p','rb'))
H0_events = pickle.load(open(f'./pre_computed/H0_DC_{pid}.p','rb'))


H0_normalized, H1_list_normalized = normalize_events(H0_events,H1_events_list,pid)

In [5]:
H1_events_list[0]

array([[3.86548143e-03, 4.11797580e-03, 4.20360624e-03, 2.94825860e-03, 1.28236982e-03, 5.34971753e-04, 1.74281422e-04, 4.72958367e-05],
       [4.78934331e-03, 3.54572716e-03, 3.82782094e-03, 2.97920719e-03, 1.55448516e-03, 8.08611477e-04, 3.95849315e-04, 3.41477437e-04],
       [6.17156101e-03, 4.80420096e-03, 5.28385646e-03, 4.66277657e-03, 3.11841911e-03, 1.66627204e-03, 8.84100844e-04, 8.40502306e-04],
       [6.80415141e-03, 5.08813869e-03, 6.74321261e-03, 7.87650547e-03, 5.99538650e-03, 2.91318874e-03, 1.42663344e-03, 1.28041093e-03],
       [5.92066408e-03, 6.55321708e-03, 1.04469843e-02, 1.23971710e-02, 9.34636049e-03, 5.08927330e-03, 1.77080723e-03, 1.16862721e-03],
       [4.67407623e-03, 7.63166183e-03, 1.13461103e-02, 1.37602885e-02, 1.05544387e-02, 5.05353897e-03, 1.83827587e-03, 7.70201670e-04],
       [4.28244312e-03, 7.45016750e-03, 1.12126923e-02, 1.25630921e-02, 8.59600725e-03, 3.31860628e-03, 1.15312184e-03, 3.29832240e-04],
       [2.98119399e-03, 4.91950732e-03, 6

In [6]:
H0_events

array([[7.29135726e-06, 7.84794061e-06, 8.45386175e-06, 7.33194935e-06, 4.86017431e-06, 3.01040042e-06, 1.62875486e-06, 7.74861451e-07],
       [7.99942110e-06, 7.39704242e-06, 8.09196130e-06, 7.19969598e-06, 5.33582636e-06, 3.88729665e-06, 2.52503307e-06, 2.10462063e-06],
       [9.16374763e-06, 8.79431057e-06, 9.60443903e-06, 8.88984728e-06, 7.25764201e-06, 5.47563591e-06, 3.93211451e-06, 3.38944648e-06],
       [1.02266949e-05, 9.59386044e-06, 1.06568527e-05, 1.10153861e-05, 9.55447814e-06, 7.12944918e-06, 5.14938894e-06, 4.50776776e-06],
       [1.09156100e-05, 1.09202233e-05, 1.26701534e-05, 1.29913920e-05, 1.13148970e-05, 8.66000144e-06, 5.74686125e-06, 4.83076337e-06],
       [9.46326679e-06, 1.06116846e-05, 1.20167449e-05, 1.31402985e-05, 1.13968686e-05, 7.97237094e-06, 5.31145612e-06, 3.81074942e-06],
       [7.52582828e-06, 9.11226827e-06, 1.09197357e-05, 1.19977889e-05, 9.89758911e-06, 6.03593924e-06, 3.63754345e-06, 2.19569829e-06],
       [5.14132419e-06, 6.57631802e-06, 7

In [6]:
sigma_a = 0.25
sigma_b = 0.04
f = 0
deltachi, best_index, H1_chisq = get_deltachi(H1_list_normalized, H0_normalized,emt_range,emm_range,0,[sigma_a,sigma_b],f,x0=[1])
#s24_cl90, s24_cl99, emm_cl90, emm_cl99 = get_contour(deltachi.T, emt_range,emm_range,2)
#best_fit_coords = (param_list[best_index]['e_mt'], param_list[best_index]['e_mm'])

In [10]:
deltachi

array([[ 0.        ],
       [12.98547712],
       [25.3298408 ],
       [36.40792578],
       [46.62030861],
       [55.2711052 ],
       [62.28192845],
       [68.1764869 ],
       [71.94664659],
       [74.16301697]])

In [20]:
from gen2.event_processing import perform_chisq
np.array([perform_chisq(H1_norm, H0_events,sigma_syst=0,z=0, sigma_a=0.25,sigma_b=0.04,sigma_g=0, x0=[1])[0] for H1_norm in H1_events_list[0:3]])

array([0.98533481, 0.98533481, 0.98533481])